# TP : Modélisation et résolution d'un problème de classification

**Instructions générales:**

- Je vous conseille de réaliser ce devoir avec `jupyter lab` et non pas `jupyter notebook`. La raison est que `jupyter notebook` a parfois du mal à afficher des images.
- Les dépendances nécessaires sont incluses dans le fichier `requirements.txt`.
- Le devoir vous a été distribué sous la forme d'un dossier contenant un notebook et d'autres fichiers. Vous pouvez déplacer le dossier où vous voulez dans votre ordinateur, mais ne changez pas la structure de ce dossier : le notebook a besoin de ces fichiers auxiliaires pour fonctionner!
- Certaines questions nécessitent de faire tourner un algorithme. Je vous recommande de ne pas faire tourner vos algorithmes plus qu'une trentaine de secondes (chez moi tout marche bien en moins d'une seconde).

**Description du devoir:**

- la partie **I** est une mise en bouche avec le problème.
- la partie **II** porte sur un problème-jouet, et est celle où vous mettrez en valeur certaines des choses que vous avez appris en cours. Elle se décompose en trois sections **indépendantes**, qui proposent de résoudre le problème de trois manières différentes. Elles peuvent être traitées à des moments différents du cours:
  - la partie **II.A** ne requiert aucune connaissance particulière du cours. Si vous voulez simplement vous amuser avec le sujet c'est par là!
  - la partie **II.B** porte sur la dualité de Rockafellar (fin du chapitre 2). Elle vous demandera de faire des petits calculs de dualité à la main. C'est l'occasion de voir si vous avez compris les bases de ce chapitre!
  - la partie **II.C** porte sur les méthodes d'éclatement, et nécéssite d'avoir vu la notion d'opérateur proximal (début du chapitre 3).
- la partie **III** est une application à un problème de traitement de données, qui consiste à exploiter les résultats de la partie **II** sur un vrai problème. C'est là que vous montrerez si vous avez compris ce qui se passe dans la partie **II**.

In [ ]:
# cette cellule importe les modules nécéssaires à ce TP. En cas d'erreur, vous devrez au préalable installer les modules manquants.
# Lors d'une première exécution cela peut prendre un peu de temps, jusqu'à 1 min parfois :(
import numpy
import matplotlib
import scipy
import sklearn

# I. Introduction

Je vous invite pour commencer à aller lire la section **"Modélisation : classification"** qui se trouve en annexe du [polycopié](https://cloud.math.univ-paris-diderot.fr/s/fLXPipwgSC9tpzj) du cours. Il y est décrit ce qu'est un problème de classification, et comment le modéliser mathématiquement.

## 1. Premier contact avec le problème

In [ ]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

**1)** On commence par importer des données

In [ ]:
X = np.load('data/donnees_entrainement.npy')

Ces données correspondent à $m$ points de $\mathbb{R}^2$, qui sont rangées dans une matrice à $m$ lignes et 2 colonnes.
Chaque *ligne* de la matrice correspond donc à un point de $\mathbb{R}^2$.
Vous pouvez regarder quelle est la taille de la matrice avec la méthode `X.shape`, ce qui vous permettra de déterminer la valuer de $m$.

In [ ]:
m = 200 # à definir

Vu que nos données sont des points du plan, on va pouvoir facilement les visualiser avec la fonction `plt.scatter` de pyplot.

In [ ]:
couleur = [None]*m # liste vide de taille m
for k in range(m):
    couleur[k] = 'blue'
_=plt.scatter(X[:, 0], X[:, 1], c=couleur)

**2)** Vous devriez être maintenant convaincus que ce jeu de données contient deux groupes de points appartenant à des familles distinctes. Or nous n'en savons rien, à priori, le seul moyen d'en être sur est d'aller regarder les *etiquettes* correspondantes. Importons-les:

In [ ]:
Y = np.load('data/etiquettes_entrainement.npy')

Vérifiez que ce vecteur ne contient que des étiquettes $\pm 1$.

En adaptant le code de la précédente question, affichez de nouveau ce nuage de points cette fois-ci avec deux couleurs: rouge pour les points avec l'étiquette $-1$, et bleu pour les points avec l'étiquette $+1$.

In [ ]:
couleur = [None]*m
for k in range(m):
    if Y[k] == 1:
        couleur[k] = 'blue'
    else:
        couleur[k] = 'red'

In [ ]:
_=plt.scatter(X[:, 0], X[:, 1], c=couleur)

**3)** Une approche intuitive pour classer ce problème consiste à tracer une droite qui va séparer ces deux nuages: ainsi fait, on pourra dire que tout nouveau point qui apparaitra d'un côté de la droite sera "rouge" et de l'autre les "bleus".

Ici on va considérer une droite dans $\mathbb{R}^2$ comme étant l'ensemble des points satisfaisant l'équation

$$ D = \{ x \in \mathbb{R}^2 \ | \ \langle a, x \rangle = b \},$$

où $a \in \mathbb{R}^2$ et $b \in \mathbb{R}$ sont à choisir. On notera par la suite $w = (a_1, a_2 ,b) \in \mathbb{R}^2 \times \mathbb{R}$ le vecteur de paramètres décrivant la droite $D$.

En utilisant le code ci-dessous, ajoutez le tracé d'une droite aux données, et jouez avec la valeur de $w$ pour essayer de trouver la droite qui sépare le mieux les deux classes.

In [ ]:
def trace_droite(w):
    # trace la droite des points x vérfiant l'équation <a,x>=b
    # où w = (a_1, a_2, b)
    # on va la tracer comme la droite alpha * t + b
    a = w[0:2]
    b = w[2]
    if a[1] == 0:
        if a[0] != 0:
            # on a une droite verticale
            x = b/a[0]
            plt.axvline(x=x)
        else: 
            return
            # a=0 donc cela ne définit pas une droite, on ne trace rien
    else:
        alpha = -a[0]/a[1]
        beta = b/a[1]
        abscisse = np.arange(-5,5,0.1)
        ordonnee = alpha*abscisse + beta
        plt.plot(abscisse, ordonnee)

In [ ]:
w = np.array([0, 1, 0]) # à modifier
trace_droite(w)
_=plt.scatter(X[:, 0], X[:, 1], c=couleur)

Pensez-vous que certaines droites sont meilleures que les autres?

## 2. Modélisation : trouver une droite séparatrice optimale

Je vous invite si ce n'est toujours pas fait à aller lire la section **"Modélisation : classification"** qui se trouve en annexe du [polycopié](https://cloud.math.univ-paris-diderot.fr/s/fLXPipwgSC9tpzj) du cours.
Il y est expliqué comment faire pour trouver un hyperplan qui sépare bien des données dans $\mathbb{R}^n$, et plus précisément comment trouver le *meilleur* hyperplan séparateur possible (en un certain sens).

Plus précisément, considérons le problème de séparer $m$ points $x_i \in \mathbb{R}^n$ avec étiquettes $y_i \in \{\pm 1\}$.
Alors on peut montrer que le meilleur hyperplan séparateur est défini par

$$ \mathcal{H}_w := \{ x \in \mathbb{R}^n \ | \ \langle a,x \rangle = b \}, \quad w=(a,b) \in \mathbb{R}^n \times \mathbb{R},$$

où $w=(a,b) \in \mathbb{R}^n \times \mathbb{R}$ est la solution du problème d'optimisation quadratique convexe suivant (appelé SVM) : 
\begin{equation}
    \min\limits_{w \in \mathbb{R}^{n} \times \mathbb{R}} \ \frac{1}{2} \Vert Pw \Vert^2 
    \quad 
    \text{ sous la contrainte que } \Phi w \preceq -e,
    \tag{P}
\end{equation}
où 

- $e:=(1,\dots,1)^\top \in \mathbb{R}^m$ 
- $P \in \mathcal{M}_{n+1}(\mathbb{R})$ est la matrice de projection sur $F = \{ w \in \mathbb{R}^{n+1} \ | \ w_{n+1}=0\}$. Autrement dit $P$ projette sur les $n$ premières coordonnées. On se convainc que $P= {\rm{diag}}(1, \dots, 1, 0)$.
- $\Phi \in \mathcal{M}_{m,n+1}(\mathbb{R})$ est la matrice définie par

\begin{equation*}
    \Phi :=
    \begin{pmatrix}
        -y_1 x_1^\top & y_1 \\
        \vdots & \vdots \\
        -y_i x_i^\top & y_i \\
        \vdots & \vdots \\
        -y_m x_m^\top & y_m
    \end{pmatrix}.
\end{equation*}

# II. Résolution du problème : phase d'entraînement

Dans cette partie, nous allons résoudre le problème (P) de trois manières différentes, dans les sections A,B et C.
Ces trois sections sont de difficulté variable, mais indépendantes:

- la section A ne requiert aucun prérequis
- la section B requiert d'avoir vu la dualité de Fenchel-Rockafellar. Elle contient notamment des petits exercices que vous êtes encouragés à faire.
- la section C requiert d'avoir vu la notion d'opérateur proximal. Elle applique un algorithme vu dans le dernier chapitre du cours.

Le but est d'écrire du code pour des données qui vivent en dimension $n$, et de les appliquer dans un premier temps à nos nuages de points pour lesquels $n=2$.
Dans la partie suivante nous appliquerons ce code à un vrai problème plus intéressant, pour lequel $n >> 2$.

**0)** On définit `e`, le vecteur $e:=(1,\dots,1)^\top \in \mathbb{R}^m$, ainsi que `Phi`, la matrice $\Phi \in \mathcal{M}_{m,3}(\mathbb{R})$ définie dans la section précédente. On rappelle que cette matrice ne dépend que de `X` et `Y`.

On définira également `infini`, le vecteur $(+\infty, \dots, +\infty) \in \mathbb{R}^m$, sachant que la quantité $+\infty$ s'obtient avec `np.inf`.

In [ ]:
e=np.ones(m)

In [ ]:
Phi = np.zeros((m,3)) # matrice vide
Phi[:, 0:-1] = -X # toutes les colonnes sauf une remplies par -X
Phi[:, -1] = e # dernière colonne remplie par des 1
Phi = np.diag(Y)@Phi # on multiplie chaque ligne par y_i

In [ ]:
infini = e*np.inf

## A. Résolution via un solveur Python

Ici nous allons simplement donner le problème à Python, qui dispose de certaines routines pour résoudre les problèmes d'optimisation. Toute la difficulté consiste en la mise en forme du problème dans une syntaxe que Python peut comprendre.

**1)** Définir une fonction `objectif` qui prend en entrée un vecteur $w=(a,b) \in \mathbb{R}^n \times \mathbb{R}$ et qui renvoie $\Vert a \Vert^2$.

**2)** On doit résoudre un problème d'optimisation sous contrainte. Pour cela, on va faire appel à la bibliothèque `scipy.optimize` qui peut faire cela pour nous. 

In [ ]:
from scipy.optimize import LinearConstraint, minimize

Tout d'abord on doit définir la contrainte $\Phi w \preceq -e$. Il faut pour cela utiliser la fonction `LinearConstraint` qui permet de définir une contrainte de la forme
$$
\{ w \in \mathbb{R}^d \ | \ a \preceq \Phi w \preceq b \}
$$
avec la commande `LinearConstraint(Phi, a, b)` (voir la [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.LinearConstraint.html#scipy.optimize.LinearConstraint) pour plus de détails). A vous de compléter la commande suivante, avec les objets définis à la question **1)**.

In [ ]:
contrainte = LinearConstraint(############)

**3)** On peut maintenant obtenir la solution de notre problème, en faisant appel à `scipy.optimize.minimize` (voir la [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) pour plus de détails).

In [ ]:
w_sol_scipy = minimize(                          # minimiser
                    fun = objectif,           # la fonction 'objectif'
                    constraints = contrainte, # sous la contrainte 'contrainte'
                    x0 = np.random.randn(3),  # en partant d'un point initial
                    ).x                       # et donne-moi le vecteur solution
w_sol_scipy

**4)** Reprendre le code de la partie **I.1**, pour afficher les nuages de points ainsi que la droite définie par le paramètre $w_{sol-scipy} \in \mathbb{R}^3$ que vous venez d'obtenir. Êtes-vous satisfait(e) de la solution obtenue?

In [ ]:
trace_droite(w_sol_scipy)
_=plt.scatter(X[:, 0], X[:, 1], c=couleur)

## B. Résolution via le problème dual et la méthode du gradient projeté

Ici nous allons nous intérésser à (D), le problème dual de (P) au sens de Fenchel-Rockafellar. Dans un premier temps nous allons calculer ce problème dual. Dans un second temps, nous allons voir qu'il a une structure plus simple, le rendant facile à résoudre.
Dans un troisième temps, nous allons résoudre (D) avec la méthode du gradient projeté, puis retrouver une solution du problème primal (P).

### 1. Calcul du problème dual

**1)** 📝 Montrer que le problème (P) peut s'écrire sous la forme 
\begin{equation*}
    \min\limits_{w \in \mathbb{R}^{n+1}} \ f(w) + g(\Phi w), 
    \tag{P}
\end{equation*}
où
- $f$ est une fonction quadratique semi définie positive,
- $g = \delta_C$ est la fonction indicatrice de $C$, un ensemble convexe fermé non vide.

**2)** 📝 D'après la dualité de Fenchel-Rockafellar, le problème dual de (P) consiste à résoudre
\begin{equation*}
    \min\limits_{u \in \mathbb{R}^m} \ f^* (-\Phi^\top u) + g^* (u), 
    \tag{D}
\end{equation*}
Vous devrez:

- Calculer la conjuguée de Fenchel $f^*$. Pour cela vous pourrez utiliser un résultat de la feuille de TD sur la conjuguée.
- Calculer la conjuguée de Fenchel $g^*$. Vous utiliserez des résultats du cours  pour montrer que 
\begin{equation*}
    g^*(u) = \delta_K(u) - \langle e, u \rangle,
\end{equation*}
où $K$ est un cône convexe fermé (très sympatique) non vide.

**3)** 📝 Conclure que le problème dual s'écrit

\begin{equation*}
    \min\limits_{u \in \mathbb{R}^M} \ h(u), 
    \quad 
    \text{ sous la contrainte que } u \in K \text{ et } \langle y,u \rangle =0,
    \tag{D}
\end{equation*}
où $h$ est une fonction convexe quadratique.

### 2. Définition de la projection



Ici on s'intéresse à la projection sur la contrainte duale
$$D:= \{ u \in \mathbb{R}^M \ | \ u \in K \text{ et } \langle y,u \rangle =0\},$$
où $K$ est le cône simple que vous avez obtenu à la section précédente.

**1)** On va ici implémenter une alternative à la méthode de projection approchée de la question précédente avec la **projection de Kiwiel**. Cette dernière s'effectue avec la fonction `proj_kiwiel` (importée ci-dessous), qui:

- projette un vecteur $u$ sur une intersection $K \cap H$ 
   - où $H$ est un hyperplan $H = \{ u \in \mathbb{R}^M \ | \ \langle y, u \rangle =0 \}$
   - où $K$ est un ensemble défini par $K = \{ u \in \mathbb{R}^M \ | \ a \preceq u \preceq b \}$
     - ici les bornes $a, b \in \mathbb{R}^M$ peuvent prendre des valeurs $\pm \infty$
- prend en entrée `proj_kiwiel(u, y, a, b)` où $u$ est le vecteur à projeter, $y$ le vecteur définissant l'hyperplan $H$, et $a,b$ sont les bornes (inférieures et supérieures) définissant $K$.
- renvoie en sortie la projection de $u$ sur $K \cap H$.

Votre travail ici consiste à définir une fonction `proj_D` qui prend en entrée un vecteur `u`, le vecteur `y`, et renvoie la projection de $u$ sur $D$.
Pour ce faire, vous définirez les bornes `a` et `b` qui définissent $K$, et vous appliquerez la projection de Kiwiel.

In [ ]:
from scripts.kiwiel import proj_kiwiel

In [ ]:
def proj_D(u, y):
    # ########  

**2) BONUS:** Dans cette question totalement facultative, on vous propose de coder vous-même une projection sur $K \cap H$ en utilisant les algorithmes vus en cours. 
En effet, on peut écrire

$$ proj_{K \cap H}(x) = {\rm{argmin}}_y  \frac{1}{2} \Vert y - x \Vert^2 + \delta_K(y) + \delta_H(y).$$

Il s'agit donc de résoudre un problème de minimisation de la forme $f + g$ où $f(y) = \frac{1}{2} \Vert y - x \Vert^2 + \delta_K(y)$ et $g(y) = \delta_H(y)$.
Ce type de problème peut se résoudre avec l'algorithme de Douglas-Rachford, qui dans ce cas particulier devient (avec un peu de travail) la méthode de **projection de Dijkstra**:
Elle se définit via des projections alternées sur $H$ et $K$, comme suit:

| Projection de Dijkstra |
|-|
| On veut projeter $u$ sur $K\cap H$ |
| On initialise $x_0=u$, et $h_0=p_0=q_0=0$, puis pour $k \geq 0$: | 
| $$\begin{cases} h_{k+1} & =  {\rm{proj}}_H(x_k+p_k) \\ p_{k+1} &= x_k + p_k - h_{k+1} \\ x_{k+1} &= {\rm{proj}}_K( h_{k+1} + q_k) \\ q_{k+1} &= h_{k+1} + q_k - x_{k+1} \end{cases}$$ |
| **Théorème:** $x_k \to {\rm{proj}}_{K\cap H}(u)$ lorsque $k \to +\infty$ | 

Nous vous proposer de coder la projection sur $K$ et $H$ (qui sont relativement faciles), puis de calculer une dizaine d'itérations de la projection de Dijkstra. Vous pourrez alors comparer votre résultat avec la projection de Kiwiel (en projetant un vecteur aléatoire par exemple), et avoir le plaisir d'avoir calculé cette projection vous-même.

### 3. Méthode du gradient projeté

La méthode du gradient projeté permet de résoudre un problème de la forme 
$$ \min f(x), x \in C$$
L'algorithme s'écrit 

| | | |
|-|-|-|
|| On choisit $x_0$ un vecteur de $\mathbb{R}^N$ et $\rho > 0$ un pas fixe. | 
|  | Pour $k\geq 0$ : $\qquad \qquad \, $   $x_{k+1}$  = ${\rm{proj}}_C(x_k - \rho \nabla f(x_k))$  | 

L'algorithme est garanti de converger si $f$ est convexe, de classe $C^1$, avec un gradient $L$-Lipschitzien, et un pas $\rho \in ]0,2/L[$.

**1)** 📝 Vérifier que les hypothèses ci-dessus sont vérifiées pour le problème dual (D). En particulier, vous calculerez $\nabla h$, et vous définirez sa constante de Lipschitz `L`.
On rappelle que la norme d'opérateur d'une matrice `A` s'obtient avec `np.linalg.norm(A,2)` ; et que la constante de Lipschitz de $\nabla f$ est la borne supérieure de $\Vert \nabla^2 f(x) \Vert$.

**2)** Définir une fonction `FB_dual` qui résoud le problème dual (D) en:

- prenant en entrée la matrice `Phi`, le vecteur `y`, un pas `rho`, un nombre d'itérations `nb_iter`
- résolvant le problème dual en appliquant la méthode du gradient projeté :
  - en initialisant les itérés en $0 \in \mathbb{R}^m$
  - avec un pas `rho`
  - en réalisant la projection sur $D$ avec une fonction définie dans la section précédente (de votre choix)
  - pendant un nombre d'itérations égal à `nb_iter`
- retournant en sortie le dernier itéré calculé 

In [ ]:
def FB_dual(Phi, y, rho, nb_iter=500):
    # #############

**3)** Trouver `u_sol_GPD` une solution approchée au problème dual (D).

**4)** 📝 D'après la caractérisation des solutions du problème primal-dual vu en cours, on sait que la solution $w$ du problème primal est liée à la solution $u$ du problème dual via l'inclusion $w \in \partial f^*(-\Phi^\top u)$.
Montrer que ceci implique que $-\Phi^\top u$ peut s'écrire   $-\Phi^\top u= (a,0)$ où $a \in \mathbb{R}^n$, et que $w=(a,b)$ pour un certain $b$ à déterminer.



**5)** Vous vérifierez numériquement que la dernière coordonnée de $\Phi^\top u$ est bien nulle. Définissez `a`.

**6)** On admet par la suite que l'on peut prendre $b=0$, c-à-d que $w = -\Phi^\top u$ est solution de notre problème primal (P). Reprendre le code de la partie **I.1**, pour afficher les nuages de points ainsi que la droite définie par le paramètre $w \in \mathbb{R}^3$ que vous venez d'obtenir. Êtes-vous satisfait(e) de la solution obtenue?

## C. Résolution via une méthode d'éclatement

Dans cette section, l'objectif est que vous soyez plus autonomes. Nous vous fournissons un algorithme, un théorème, et vous devez vous en servir pour trouver une solution aux problèmes (P) et (D). A vous de vérifier que votre code fonctionne et vous fournit une solution satisfaisante.

On introduit l'algorithme de **Loris-Verhoeven**, qui est une généralisation de la méthode du gradient proximal, et un cas particulier de l'algorithme de Yan, qui s'applique à la minimisation d'une fonction de la forme $g(Ax) + h(x)$, où $g \in \Gamma_0(\mathbb{R}^M)$ et $h \in \Gamma_0(\mathbb{R}^N) \cap C^{1,1}_L(\mathbb{R}^N)$:

| L'algorithme de **Loris-Verhoeven** |
| --- |
| Soient $x_0 \in \mathbb{R}^N, u \in \mathbb{R}^M$, $\lambda, \sigma >0$ |
| $$\begin{cases} x_{k+1} &= x_k - \lambda \nabla h(x_k) - \lambda A^\top u_n \\ u_{k+1} &=  {\rm prox}_{{\sigma}g^*} \left(u_n + \sigma A \left[2x_{k+1} - x_k + \lambda \nabla h(x_k) - \lambda \nabla h(x_{k+1}) \right]\right) \end{cases}$$ |

**Théorème:** Soient $h : \mathbb{R}^n \to \mathbb{R}$ une fonction différentiable à gradient $L$-Lipschitzien,  $g : \mathbb{R}^m \to \mathbb{R}\cup\{+\infty\}$ une fonction convexe s.c.i. propre, et $A \in \mathcal{M}_{m,n}(\mathbb{R})$.
Soit $(x_k,u_k)_{k \in \mathbb{N}}$ une suite générée par l'algorithme de Loris-Verhoeven, avec
$$ \lambda \in \left]0, \frac{2}{L}\right[ \quad \text{ et } \quad \sigma \in \left]0, \frac{1}{\lambda \Vert A \Vert^2}\right].$$
Alors 
- $x_k$ converge vers un minimiseur de $f + g \circ A$, s'il en existe,
- $u_k$ converge vers un minimiseur de $f^* \circ -A^\top + g^*$ s'il en existe.

## D. Conclusion

Comparer les trois solutions obtenues. D'une part vous visualiserez les 3 droites séparatrices correspondantes. Ensuite vous évaluerez $f$ en ces points. Donner votre avis sur vos résultats.

# III. Classifier des images

## 1. Obtention des données : Importer le jeu de données MNIST

On va résoudre un problème similaire, sauf que cette fois les données ne sont plus des points $x_i \in \mathbb{R}^2$ rouges ou bleus, mais des images $x_i \in \mathbb{R}^{64}$ de chiffres écrits à la main.
On ne parlera donc plus de "droite séparatrice" mais d'"hyperplan séparateur".

| Un nouveau jeu de données | 
| ----------- |
| ![](images/mnist.jpg) |
| Chaque image de $8\times 8$ pixels est représentée par un point $x_i \in \mathbb{R}^{64}$ |

Pour ce TP on va se contenter de séparer des images de 0 et de 1. Leurs étiquettes $y_i$ prendront respectivement les valeurs $-1$ et $+1$.



In [ ]:
import sklearn.datasets
import sklearn.model_selection

In [ ]:
# paramètres pour l'affichage des images de nombres
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['ytick.labelleft'] = False
plt.rcParams['image.cmap'] = 'gray_r'

In [ ]:
# importe les données
digits = sklearn.datasets.load_digits() # Importe un jeu de données à classer (10 classes)
digits.data = digits.data*1/np.max(digits.data) # normalise : coeffs dans [0,1]
classes = [0, 1] # Définit les 2 classes de nombres avec lesquelles on va travailler
idx_classes = np.logical_or(digits.target == classes[0], digits.target == classes[1]) # Localise les deux classes ..
digits.data = digits.data[idx_classes] # .. les extrait ..
digits.target = digits.target[idx_classes] # .. et jette le reste
digits.target = np.where(digits.target==classes[0],-1, 1) # Transforme les étiquettes de classes en {-1,+1}

X, X_test, Y, Y_test = sklearn.model_selection.train_test_split( # On coupe le jeu de données en deux
                        digits.data, digits.target, train_size=25, shuffle=True)

In [ ]:
# Affiche les 25 premières images de X
fig, axs = plt.subplots(5, 5, figsize=(3, 3))
for i in range(5):
    for j in range(5):
        k = i*5 + j
        _ = axs[i,j].imshow(X[k].reshape(8,8))
plt.subplots_adjust(wspace=0, hspace=0)

**1)** On dispose de données `X`, qui contient des images de 0 et 1 écrits à la main. Plus précisément, pour tout `k`, `X[k]` représente une telle image 2D (8x8 pixels) qui a été aplatie en un vecteur 1D.

Utiliser `.shape` pour déterminer le nombre d'images que contient `X`.

Prendre une image au hasard, et l'afficher avec la fonction `plt.imshow()`. Afin de l'afficher, vous aurez besoin de temporairement remettre cette image sous forme 2D avec la méthode `.reshape(nb_lignes, nb_colonnes)` . Est-ce l'image d'un 0 ou d'un 1?

In [ ]:
X.shape

In [ ]:
plt.imshow(X[10].reshape(8,8))

**2)** On dispose d'étiquettes `Y` qui encodent la nature des images contenues dans `X`. Plus précisément, `Y[k]` contient `-1` si `X[k]` est l'image d'un 0, ou `+1` si `X[k]` est l'image d'un 1.

Reconsidérer l'image affichée à la question précédente, et vérifier que son étiquette correspond à ce que vous avez observé.

In [ ]:
Y[10]

## 2. Phase d'entraînement : Trouver un classifieur avec la méthode SVM

**0)** Définir `m` le nombre de données dans notre jeu de données ; et `n` la taille de chacune de ces données.

In [ ]:
m = X.shape[0]
n = X.shape[1]

**1)** Nous allons trouver un classifieur linéaire avec la méthode SVM, comme au I.

Pour ce faire, définir deux vecteurs `e`$= (1, \dots , 1)^\top$, `infini`$=(+\infty, \dots, +\infty)^\top$ de $\mathbb{R}^m$, ainsi que la matrice $\Phi \in \mathcal{M}_{m,n+1}(\mathbb{R})$ définie en Section I.2. 

In [ ]:
e = np.ones(m)

In [ ]:
Phi = np.zeros((m, n+1)) # matrice vide
Phi[:, 0:-1] = -X # toutes les colonnes sauf une remplies par -X
Phi[:, -1] = e # dernière colonne remplie par des 1
Phi = np.diag(Y)@Phi # on multiplie chaque ligne par y_i

In [ ]:
infini = e*np.inf

**2)** Adapter à partir de la Section **II** la méthode de résolution de votre choix pour obtenir une solution $w=(a,b) \in \mathbb{R}^{n+1}$ du problème de SVM (P). Vous appellerez cette solution `w_mnist`

**3)** Ecrire une fonction `classifieur`, qui:
- prend en entrée une image aplatie `x`$\in \mathbb{R}^n$
- prend en entrée un vecteur de paramètres  `w`$=(a,b)\in \mathbb{R}^n \times \mathbb{R}$
- retourne +1 si $\langle a, x \rangle \geq  b$, -1 si $\langle a, x \rangle \leq b$

In [ ]:
def classifieur(x, w):
    # ###########

**4)** Prendre une image au hasard dans `X`, et comparer sa vraie étiquette avec la prédiction faite par notre nouveau classifieur. Notre prédiction est elle bonne?

## 3. Phase de test : connaitre la vraie performance de notre classifieur

On vient de voir que notre classifieur marche bien lorsque on l'applique aux images contenues dans `X`. Or ceci n'est pas très surprenant : le classifieur a été construit à partir de `w_mnist`, la solution d'un problème d'optimisation dépendant des données contenues dans `X,Y`. Pour vraiment déterminer si notre modèle a **appris** quelque chose, il faut le tester sur des données qu'il n'a encore **jamais vues**.

On va donc maintenant utiliser les données de test `X_test` et `Y_test` que l'on a pas encore utilisées.

**1)** Calculer le pourcentage de bonnes réponses données par notre `classifieur`. Autrement dit, vous allez parcourir l'ensemble des données de test, et calculer le pourcentage du nombre de fois que le classifieur donne la bonne prédiction, en la comparant à la vraie étiquette contenue dans `Y_test`. Que pensez-vous du nombre obtenu?

**2)** Si le taux de bonne réponse n'est pas de 100%, essayez de trouver dans le jeu de données quelles sont les images sur lesquelles le classifieur se trompe.

**3)** Si vous retournez au début de la section **III.**, vous pouvez voir qu'au moment de l'importation nous avons demandé à travailler avec les `classes` 0 et 1.

Remplacez ces chiffres par deux autres chiffres de votre choix, et relancez votre code afin de déterminer le taux de bonne réponse du classifieur. Essayez de prendre des chiffres difficiles à classer!

**NB:** Vous ne serez pas notés sur cette question, qui vous laisse libre de vous amuser. Néanmois je vous conseille de ne pas l'ignorer, car vous devrez de toute façon faire ce genre de choses dans les questions suivantes.

## 4. Aller plus loin : Classification multiple

### a. Discussion

On a vu comment classer des données à deux étiquettes. Mais en pratique il y a souvent un plus grand nombre de classes : par exemple MNIST peut contenir jusqu'à 10 classes: les chiffres de 0 à 9! Un tel classifieur vous est par exemple mis à disposition sur [ce site](https://mco-mnist-draw-rwpxka3zaa-ue.a.run.app/), qui vous permet de dessiner en ligne un chiffre et vous fournira en temps réel une estimation de la probabilité d'appartenance à une classe de chiffre.

Je vous propose dans cette section de construire un tel classifieur. Notre stratégie va consister à réunir plusieurs classifieurs à deux classes (que vous avez appris à construire dans la section précédente) pour construire un classifieur à 10 classes. 
Plus précisément notre stratégie sera:

- Couper le jeu de données en deux classes : les 0 et le reste (1, ..., 9). On produit alors un classifieur qui sera capable de savoir si une image est un zéro, ou non.
- On refait la même chose en isolant cette fois 1 versus le reste (0,2, ..., 9). Et ainsi de suite. Ce qui nous donnera 10 classifieurs, chacun répondant à la question "est-ce que cette image est un 0? un 1? etc.
- Etant donné une nouvelle image, on la passe dans les 10 classifieurs, et en fonction des 10 prédictions on prend une décision.




### b. Manipulation des données et construction du premier classifieur

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.model_selection
from scipy.optimize import LinearConstraint, minimize

In [ ]:
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['xtick.labelbottom'] = True
plt.rcParams['ytick.labelleft'] = True
plt.rcParams['image.cmap'] = 'gray_r'

In [ ]:
digits = sklearn.datasets.load_digits() # Importe un jeu de données à classer (10 classes)
digits.data = digits.data*1/np.max(digits.data) # normalise : coeffs dans [0,1]
X, X_test, Y, Y_test = sklearn.model_selection.train_test_split( # On coupe le jeu de données en deux
                        digits.data, digits.target, train_size=0.25, shuffle=True)
etiquettes_original = Y

**1)** Déterminer `m` le nombre de données contenues dans `X`, et `n` la dimension de ces données.

In [ ]:
m = X.shape[0]
n = X.shape[1]

**2)** Vérifier que `Y` contient bien des étiquettes allant de 0 à 9. Dans le but de classer les 0 vs. le reste, créez un nouveau vecteur d'étiquettes `Y_temp` qui vaut +1 pour les 0, et -1 pour le reste. Faites bien attention à ne pas modifier le `Y` original!

In [ ]:
# 0 vs le reste
cible = 0
Y_temp = np.where(etiquettes_original==cible, 1, -1) # Transforme 'cible' en +1, le reste en -1

**3)** En vous inspirant de ce que vous avez fait à la section précédente:

- définissez le problème de SVM associé au problème de classifier 0 vs. le reste
- résolvez-le, afin d'obtenir `w` un vecteur de paramètres définissant un hyperplan qui sépare les 0 des autres chiffres

**4)** Prenez quelques examples dans `X` et vérifiez que `w` définit un hyperplan qui sépare bien les 0 du reste (cf. section précédente). Rappelez-vous que les vraies étiquettes sont contenues dans `Y`.

### c. Construction d'un classifieur général

**1)** Ici vous devrez reproduire ce que vous venez de faire :

- Pour chaque chiffre 'cible' entre 0 et 9:
  - Définir un vecteur d'étiquettes décrivant un problème de classification 'cible' vs. le reste
  - Définir le problème SVM associé et le résoudre, ce qui vous donnera un vecteurs de paramètres w
  - Ranger chacun de ces vecteurs $w \in \mathbb{R}^{n+1}$ comme ligne d'une matrice $W \in \mathcal{M}_{10, n+1}(\mathbb{R})$

**2)** Vérifiez que la matrice `W` obtenue donne de bons résultats. Pour cela, pous pourrez prendre une image quelconque, et la passer dans le classifieur binaire pour chaque ligne de `W` : il devrait renvoyer +1 seulement pour le bon indice.

**3)** Définir une fonction `classifieur_general` qui:

- prend en entrée une donnée $x$ et une matrice de paramètres  $W \in \mathcal{M}_{10, n+1}(\mathbb{R})$
- pour chaque chiffre `i` entre 0 et 9, utilise le classifieur binaire induit par `W[i,:]` pour déterminer si $x$ est égal à `i` ou non
- renvoie `i` si $x$ est égal à `i`
- pensez à faire en sorte que la fonction renvoie toujours quelque chose

Vous testerez sur un exemple que cela marche bien.

In [ ]:
def classifieur_general(x, W):
    # #########

**4)** Appliquez votre `classifieur_general` aux données de test, comme au **II.3.1)**. Quel est le pourcentage de bonnes réponses de votre classifieur? Que pensez-vous du résultat?

### d. Construction d'un classifieur général amélioré



Le classifieur précédent souffre de quelques défauts.
Par exemple, si une image est classée comme appartenant à deux classes, comment choisir laquelle des deux est la meilleure?
Et si une image apparait classée comme appartenant à aucune classe, comment choisir quelle classe est la moins mauvaise?
On donc besoin que les classifieurs binaires renvoient un peu plus que une réponse $\pm 1$ pour chaque chiffre : il nous faut également un indice de confiance, une *probabilité* que la réponse $\pm 1$ soit correcte.

Par exemple, imaginons que le classifieur biniaire "0 vs. le reste" nous dise que $\mathbb{P}(x=0) = 0.4$. Dans ce cas on pourrait conclure que $x$ n'est pas un 0, puisque $\mathbb{P}(x\neq 0) = 0.6$. Mais si tous les autres classifieurs binaires nous disent également que $\mathbb{P}(x=k) = 0.01$, alors on pourrait se dire que 0 est la moins mauvaise réponse.

Pour notre problème, cette probabilité va être liée à la distance entre la donnée $x$ et l'hyperplan séparateur $H_w$ : plus la donnée est proche de l'hyperplan, et moins on aura confiance en la prédiction, donc plus basse sera la probabilité.

**1)** Définir une fonction `classifieur_general2` qui reprend le principe de `classifieur_general`. Cette fois-ci, l'indice `i` renvoyé sera celui qui maximise la probabilité $p_i$, où $p \in \mathbb{R}^{10}$ est défini par :

$$ p_i := \frac{e^{v_i}}{\sum\limits_{j=1}^p e^{v_j}},
\quad
v_i := \langle a^i, x \rangle + b^i
$$
avec `W[i,:]`$=(a^i, b^i)$. 
Vous testerez sur un exemple que votre fonction marche bien.

**2)** Estimer la performance de votre nouveau classifieur sur les données de test. Comparer avec le classifieur précédent. 

**Pour aller plus loin:** 

- Rien ne vous empêche de dessiner vos propres chiffres, et de tester si votre classifieur le reconnait.. Pour cela il vous suffit de produire une image carrée, de la redimensionner en image 8x8, puis de l'importer dans python.

**Notes**

Ressources utilisées pour le chargment et utilisation de MNIST:

- https://dmkothari.github.io/Machine-Learning-Projects/SVM_with_MNIST.html
- https://towardsdatascience.com/support-vector-machine-mnist-digit-classification-with-python-including-my-hand-written-digits-83d6eca7004a
- https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html